# Locally Linear Embedding(LLE)

LLE (Locally Linear Embedding) is a technique used to reduce the number of dimensions in a dataset without losing the important shape or structure of the data. It is an unsupervised method meaning it works without needing labeled data.

### How is LLE different from PCA and PDA?
| Method                                 | Type                | What it Preserves?                                                                      | When Useful?                                                        |
| -------------------------------------- | ------------------- | --------------------------------------------------------------------------------------- | ------------------------------------------------------------------- |
| **PCA** (Principal Component Analysis) | Linear              | Global variance (maximizes variance directions)                                         | Good for compression, noise reduction, linear data                  |
| **LDA** (Linear Discriminant Analysis) | Linear (supervised) | Class separability (maximizes distance between classes, minimizes intra-class variance) | Classification tasks with labels                                    |
| **LLE** (Locally Linear Embedding)     | Nonlinear           | Local geometry of neighborhoods                                                         | Nonlinear manifolds, visualization, unsupervised feature extraction |

## Quick Comparison Recap

- PCA: Keeps the directions with biggest variance (spread).

- LDA: Keeps the directions that best separate groups (supervised).

- LLE: Keeps the neighborhood relationships on curved manifolds (nonlinear).

## Math

### Goal: 
Reconstruct 𝑥𝑖 from its 𝑘 nearest neighbors using weights 𝑤𝑖j that sum to 1 (affine constraint). This preserves local (neighborhood) shape.

### Step 1
- Find neighbors
- Look at the closest 𝑘 points.

In [43]:
import numpy as np
from scipy.spatial.distance import cdist

# Given points
points = np.array([
    [0, 0],      # X1
    [2, 0],      # X2
    [0, 3],      # X3
    [1, 1.5],    # X4
    [0.2, 0.5]   # X5
])

k = 2  # number of neighbours

In [44]:
# Step 1: Compute pairwise Euclidean distances
dist_matrix = cdist(points, points, metric='euclidean')
print(dist_matrix)

[[0.         2.         3.         1.80277564 0.53851648]
 [2.         0.         3.60555128 1.80277564 1.86815417]
 [3.         3.60555128 0.         1.80277564 2.50798724]
 [1.80277564 1.80277564 1.80277564 0.         1.28062485]
 [0.53851648 1.86815417 2.50798724 1.28062485 0.        ]]


In [45]:
# Step 2: Identify k-nearest neighbours for each point
neighbors = {}
for i in range(len(points)):
    sorted_idx = np.argsort(dist_matrix[i])
    neighbors[i] = sorted_idx[1:k+1]  # skip itself (index 0 in sorted list)

print("Nearest neighbors for each point:")
for i, nbrs in neighbors.items():
    print(f"X{i+1}: {[f'X{j+1}' for j in nbrs]}")

Nearest neighbors for each point:
X1: ['X5', 'X4']
X2: ['X4', 'X5']
X3: ['X4', 'X5']
X4: ['X5', 'X1']
X5: ['X1', 'X4']


### Step 2: Building Covariance Matrix

For each point 𝑥𝑖:

- We have already chosen its 𝑘 nearest neighbors.
- Now we “shift” those neighbors so that 𝑥𝑖 itself is at the origin.
- This means subtracting 𝑥𝑖 from each neighbor: 
  
𝑧𝑗 = 𝑥𝑗 − 𝑥𝑖
  
- Stack these shifted vectors into a matrix 𝑍.
- Z has shape (𝐷,𝑘), where 𝐷 is the dimension (2 here) and 𝑘 is the number of neighbors. Each column of 𝑍 is one shifted neighbor.

C=Z⊤Z.
  
It’s called the Gram matrix or local covariance matrix in LLE. Each entry in 𝐶 is just a dot product between two shifted neighbor vectors.

In [ ]:
for i in range(len(points)):
    xi = points[i]
    nbr_idx = neighbors[i]
    nbr_points = points[nbr_idx]

    # Shifted neighbors
    Z = (nbr_points - xi).T  # shape (2, k)

    # Local covariance matrix
    C = Z.T @ Z

    # Regularization (to avoid singular matrix issues)
    # C += 1e-3 * np.trace(C) * np.eye(k)

    # Solve for weights
    ones = np.ones(k)
    w = np.linalg.solve(C, ones)
    w = w / np.sum(w)  # normalize to satisfy sum(w)=1

    all_weights[f"X{i+1}"] = (nbr_idx, w)

    # Verification
    xi_reconstructed = np.dot(w, nbr_points)
    error = np.linalg.norm(xi - xi_reconstructed)

    print(f"\nPoint X{i+1}:")
    print(f"  Neighbors: {[f'X{j+1}' for j in nbr_idx]}")
    print(f"  Weights: {w}")
    print(f"  Reconstructed: {xi_reconstructed}")
    print(f"  Original: {xi}")
    print(f"  Error: {error:.6f}")


Point X1:
  Neighbors: ['X5', 'X4']
  Weights: [ 1.40243902 -0.40243902]
  Reconstructed: [-0.12195122  0.09756098]
  Original: [0. 0.]
  Error: 0.156174

Point X2:
  Neighbors: ['X4', 'X5']
  Weights: [0.57317073 0.42682927]
  Reconstructed: [0.65853659 1.07317073]
  Original: [2. 0.]
  Error: 1.717911

Point X3:
  Neighbors: ['X4', 'X5']
  Weights: [ 1.42682927 -0.42682927]
  Reconstructed: [1.34146341 1.92682927]
  Original: [0. 3.]
  Error: 1.717911

Point X4:
  Neighbors: ['X5', 'X1']
  Weights: [ 3.27586207 -2.27586207]
  Reconstructed: [0.65517241 1.63793103]
  Original: [1.  1.5]
  Error: 0.371391

Point X5:
  Neighbors: ['X1', 'X4']
  Weights: [0.70769231 0.29230769]
  Reconstructed: [0.29230769 0.43846154]
  Original: [0.2 0.5]
  Error: 0.110940
